In [ ]:
import pandas as pd, 

In [3]:
url = "https://raw.githubusercontent.com/nkmwicz/worldcup2018data/refs/heads/main/cleaned_events_world_cup2018.csv"
df = pd.read_csv(url)
df.head()

,eventId,subEventName,tags,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,x1,y1,x2,y2
0,8,Simple pass,['Accurate'],Mohammad Ibrahim Al Sahlawi,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,1.656214,Simple pass,258612104,50,50,35.0,53.0
1,8,High pass,['Accurate'],Abdullah Ibrahim Otayf,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,4.487814,High pass,258612106,35,53,75.0,19.0
2,1,Air duel,"['Won', 'Accurate']",Ilya Kutepov,"Russia - Saudi Arabia, 5 - 0",Duel,Russia,1H,5.937411,Air duel,258612077,25,81,37.0,83.0
3,1,Air duel,"['Lost', 'Not accurate']",Yasir Gharsan Al Shahrani,"Russia - Saudi Arabia, 5 - 0",Duel,Saudi Arabia,1H,6.406961,Air duel,258612112,75,19,63.0,17.0
4,8,Simple pass,['Accurate'],Salman Mohammed Al Faraj,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,8.562167,Simple pass,258612110,63,17,71.0,15.0


In [4]:
# group by matchId and teamId
group_df = df.groupby(['matchId', 'teamId']).agg(
    passes = ("is_pass", "sum"),
    goals = ("is_goal", "sum"),
).reset_index()
group_df

KeyError: "Column(s) ['is_goal', 'is_pass'] do not exist"

In [ ]:
# get r, p and r_sq
def create_regplot(df: pd.DataFrame, x:str, y:str) -> None:
    x_data= df[x]
    y_data = df[y]
    r, p = stats.pearsonr(x_data, y_data)
    r_sq = r**2
    text=f"r-val={r:.3f}\np-val={p:.3f}\nr-sq={r_sq:.3f}"
    reg = sns.regplot(
        data=df,
        x=x, 
        y=y,
        scatter_kws = ("color": "blue", "aplha": 0.5),
        line_kws = ("color": "red", "label": "line of the best fit")     
    )
    plt.text(0.15, 0.70, text, transform=plt.gcf().transfigure)
    reg.set(
        xlabel = x.replace("_", " ").title(),
        ylabel = y.replace("_", " ").title(),
    )
    plt.show()

SyntaxError: invalid syntax (994197083.py, line 12)

In [ ]:
create_regplot(df=group_df, x="passes", y="goals")

In [ ]:
adf = df.loc[df['x2']>66]
#adf = df.loc[df['x2']>33]
adf
df['shot'] = df['eventName'].apply(lambsa x: x=="Shot").astype(int)

,eventId,subEventName,tags,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,x1,y1,x2,y2
1,8,High pass,['Accurate'],Abdullah Ibrahim Otayf,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,4.487814,High pass,258612106,35,53,75.0,19.0
4,8,Simple pass,['Accurate'],Salman Mohammed Al Faraj,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,8.562167,Simple pass,258612110,63,17,71.0,15.0
5,8,Simple pass,['Accurate'],Salem Mohammed Al Dawsari,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,10.991292,Simple pass,258612113,71,15,92.0,11.0
6,1,Ground attacking duel,"['Free space right', 'Won', 'Accurate']",Salman Mohammed Al Faraj,"Russia - Saudi Arabia, 5 - 0",Duel,Saudi Arabia,1H,14.081637,Ground attacking duel,258612115,92,11,88.0,11.0
7,8,Simple pass,['Accurate'],Salman Mohammed Al Faraj,"Russia - Saudi Arabia, 5 - 0",Pass,Saudi Arabia,1H,15.039163,Simple pass,258612116,88,11,88.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101746,1,Ground defending duel,"['Neutral', 'Accurate']",Ivan Peri\u0161i\u0107,"France - Croatia, 4 - 2",Duel,Croatia,2H,2919.788786,Ground defending duel,263885655,94,23,100.0,100.0
101748,1,Air duel,"['Won', 'Accurate']",Nabil Fekir,"France - Croatia, 4 - 2",Duel,France,2H,2957.224360,Air duel,263885694,56,89,73.0,82.0
101751,8,High pass,['Not accurate'],Domagoj Vida,"France - Croatia, 4 - 2",Pass,Croatia,2H,2964.715715,High pass,263885650,35,22,74.0,23.0
101755,7,Touch,[],Ivan Peri\u0161i\u0107,"France - Croatia, 4 - 2",Others on the ball,Croatia,2H,2979.084611,Touch,263885653,64,6,82.0,2.0


In [ ]:
group_adf = adf.groupby(['matchId', 'teamId']).agg(
    passes = ("is_pass", "sum"),
    goals = ("is_goal", "sum"),
).reset_index()
group_adf
create_regplot(group_adf, "passes", "goals")

In [ ]:
group_adf = df.groupby(['matchId', 'teamId']).agg(
    shots = ("shot", "sum"),
    goals = ("is_goal", "sum"),
).reset_index()
group_adf
create_regplot(group_adf, "shots", "goals")